2.2.1 Observaciones de AEMET
=============================
Este es el primero notebook dedicado a generar la climatología de temperatura en España.<br>
El primer paso será familiarizarse con como establecer comunicación con la API de AEMET para poder descargarse datos, en este caso de observación de las todas estaciones.<br>
La libreria de Python que se encargará de establecer el contacto con todo tipo de peticiones a través de la red es **requests**

In [ ]:
import requests

Para accedeer a los datos en formato JSON es necesario importar la libreria correspondiente **json**<br>
Por otro lado, para posteriormente exportar los resultados a un fichero en formato CSV importaremos las librerias **pandas** y **datetime**

In [25]:
import json
import pandas as pd
import datetime

# Acceso datos AEMET
El acceso a los datos se hace en 2 pasos. El primero revisa la peticion.
El status de la peticion aparece en la key 'descripcion'. Si el resultado es 'exito', todo ha ido bien en la key 'datos' aparece la URL donde se encuentran esos datos.
Por eso es necesario volver a realizar una segunda peticion
El resultado de esta segunda peticion es un fichero JSON con los datos deseados

In [24]:
# URL de OpenData de AEMET
url = "https://opendata.aemet.es/opendata/api/observacion/convencional/todas"

# Aqui la APIKEY de AEMET
api_key = "eyJhbGciOiJIUzI1NiJ9.eyJzdWIiOiJtaWd1ZWwuY29yZG9iYUB3ZWF0aGVydHJlbmQuZXMiLCJqdGkiOiI5ODZhMT"+\
"JlYy03OGViLTQwOTktYmIxZS1hNmI4ZjQ3OTg0MzMiLCJpc3MiOiJBRU1FVCIsImlhdCI6MTUxNzg0OTI0MywidXNlcklkIjoiOT"+\
"g2YTEyZWMtNzhlYi00MDk5LWJiMWUtYTZiOGY0Nzk4NDMzIiwicm9sZSI6IiJ9.YeSmzO_si0SavN2KYCKzjnsSd_NzNtDqOZP9nMQgYA0"

querystring = {"api_key":api_key}

headers = {'cache-control': "no-cache"}

# La web de AEMET no tiene certificado digital
# Para evitar mensaje de 'warning' al establecer una conexión insegura es necesario ignorar esos mensajes
import warnings
warnings.filterwarnings('ignore')

In [27]:
# Establecemos la primera conexión
response = requests.get(url, headers=headers, params=querystring,verify=False)
RESPONSE = eval(response.text)

# Usamos try/except para conocer algo mas de los posibles errores
try:    
    if RESPONSE['descripcion'] == 'exito':
        # Si la operacion ha sido un exito, establecemos la segunda conexion
        acceso = requests.get(RESPONSE['datos'], headers=headers, params=querystring, verify=False)
    else:
        print ('Error al establecer la segunda conexion : al acceder al fichero JSON')
except:
    print ('Error al establecer la primera conexion')

# Utilizamos Pandas para pasar sencillamente de JSON a CSV
DATOS = pd.DataFrame.from_dict(acceso.json())    


In [28]:
# Revisamos el contenido de los datos
DATOS.head()

,alt,dmax,dmaxu,dv,dvu,fint,geo700,geo850,geo925,hr,...,tpr,ts,tss20cm,tss5cm,ubi,vis,vmax,vmaxu,vv,vvu
0,32.0,281.0,NaN,286.0,NaN,2019-01-09T13:00:00,NaN,NaN,NaN,52.0,...,4.1,NaN,NaN,NaN,VANDELLÓS,NaN,21.1,NaN,7.8,NaN
1,71.0,290.0,NaN,280.0,NaN,2019-01-09T13:00:00,NaN,NaN,NaN,49.0,...,4.3,16.0,NaN,NaN,REUS/AEROPUERTO,30.0,20.6,NaN,11.9,NaN
2,233.0,NaN,NaN,NaN,NaN,2019-01-09T13:00:00,NaN,NaN,NaN,45.0,...,NaN,NaN,NaN,NaN,VALLS,NaN,NaN,NaN,NaN,NaN
3,55.0,307.0,NaN,305.0,NaN,2019-01-09T13:00:00,NaN,NaN,NaN,47.0,...,NaN,NaN,NaN,NaN,TARRAGONA FAC. GEOGRAFIA,NaN,15.9,NaN,5.5,NaN
4,632.0,312.0,NaN,304.0,NaN,2019-01-09T13:00:00,NaN,NaN,NaN,45.0,...,NaN,NaN,NaN,NaN,PONTONS,NaN,13.9,NaN,7.7,NaN


In [29]:
# La funcion 'numdate' convierte un string 'datein' en un tipo de dato 'datetime' a partir de un formato de fecha 'formatin'
def numdate(datein,formatin):
    dateout = datetime.datetime.strptime(datein,formatin)
    return dateout

# La funcion 'write_date' convierte un tipo de dato 'datetime' a string a partir de un formato de fecha 'formatout'
def write_date(datein,formatout):
    dateout = datein.strftime(formatout)
    return dateout

In [30]:
# Usamos las funcion 'numdate' para tener la fecha inicial (.min) y final (.max) en formato 'datetime'
last_time  = numdate(DATOS['fint'].max(),'%Y-%m-%dT%H:%M:%S')
first_time = numdate(DATOS['fint'].min(),'%Y-%m-%dT%H:%M:%S')

In [31]:
# Usamos la funcion 'write_date'para escribir las fechas inicial y final en el formato deseado 'dt_format_out'
dt_format_out = '%Y%m%dT%H'

# Nombre del fichero
file_csv = 'aemet_observacion_todas_%s_%s.csv' % (write_date(first_time,dt_format_out),
                                                  write_date(last_time,dt_format_out))
# Accion final, exportamos el DataFrame a CSV
DATOS.to_csv(file_csv,index=False)